In [ ]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

# Testes Modulares

### Geração de Massa

In [ ]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'etapa':['BUREAU CADASTRAL'],
                             'flagErroReceitaFederal':[False],
                             'flagErroCredilink':[False],
                             "marcacaoMesa":["XPTO"],
                             'solicitante.credilink.obito':["NAO"],
                             'solicitante.credilink.situacaoDocumento':["regular"]
                             })

In [ ]:
# Teste de casos de erro na chamada de origens
massa_validaDadosCadastrais1 = sherlock.Gera_Massa(base_entrada)
massa_validaDadosCadastrais1.gera_cenario('etapa',['BUREAU CADASTRAL'])
massa_validaDadosCadastrais1.gera_cenario('flagErroReceitaFederal',[False]) # a regra de erro bacen é executada no decision flow, testaremos no e2e
massa_validaDadosCadastrais1.gera_cenario('flagErroCredilink',[False]) # a regra de erro bacen é executada no decision flow, testaremos no e2e
massa_validaDadosCadastrais1.gera_cenario('marcacaoMesa',["XPTO","MESA"]) 
massa_validaDadosCadastrais1.gera_cenario('solicitante.credilink.obito',["NAO","SIM","-99999"]) 
massa_validaDadosCadastrais1.gera_cenario('solicitante.credilink.situacaoDocumento',["regular","xyz","-99999"]) 

massa_validaDadosCadastrais1 = massa_validaDadosCadastrais1.get_massa_final()

massa_validaDadosCadastrais = sherlock.Gera_Massa(base_entrada)
massa_validaDadosCadastrais.empilha_massas([
                                           massa_validaDadosCadastrais1
                                           ])
       
massa_validaDadosCadastrais.set_bom_arqv()
massa_validaDadosCadastrais.converte_massa()

### Execução Modular e Geração de CSV

In [ ]:
#Execução modular
listaResultado = rp.execucaoModular(massa_validaDadosCadastrais.get_massa_json(),rp.validaDadosCadastrais)

In [ ]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['passouDadosCadastrais'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)

freqzinha = freqzinha.sort_values('valor', ascending=True)
freqzinha

In [ ]:
# Renomeação das colunas para o cabeçalho do RMA
massa_validaDadosCadastrais.rename_cabecalho()

#Contagem da qtd de regras negativas em que o cliente se enquadrou
tam = 0
for x in listaResultado:
    t = len(x['payloadHomol']['saidas']['regrasNegativas'])
    x['payloadHomol']['saidas']['qtdNegativas'] = t
    if t > tam:
        tam = t

print("tam: "+str(tam))

for y in range (0,tam):
    massa_validaDadosCadastrais.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".nomeRegra","regrasNegativas#"+str(y+1)+".nomeRegra")
    massa_validaDadosCadastrais.atribui_expected(listaResultado,"payloadHomol.saidas.regrasNegativas."+str(y)+".descricao","regrasNegativas#"+str(y+1)+".descRegra")

massa_validaDadosCadastrais.atribui_expected(listaResultado,'payloadHomol.saidas.statusDecisao','statusDecisao')
massa_validaDadosCadastrais.atribui_expected(listaResultado,'payloadHomol.saidas.vlrLimiteAprovado','vlrLimiteAprovado')
massa_validaDadosCadastrais.atribui_expected(listaResultado,'payloadHomol.saidas.qtdNegativas','versionamentoPolitica')

massa_validaDadosCadastrais.get_massa_dm()

### Conversão da massa para CSV

In [ ]:
massa_validaDadosCadastrais.get_massa_dm().to_csv('C:/Users/T51657/Desktop/Homologação/Modelos Online/v2/massas/Modulares/massaValidaDadosCadastrais.csv',index=False)

#